Here we estimate the following conditional median function:
$$
\text{med}[valuation_i|ispolice_i]=\alpha+\beta_1 ispolice_i,
$$
using the entire sample where the number of bids received is strictly between 3 and 12.

In [1]:
import sys
import pandas as pd
import re
 
sys.path.append('../')

from functions import *

df = pd.read_csv("../../data/simulated.csv")

def comma(s):
    add_comma = lambda match: match.group(0).replace(" ", ", ")
    
    s = s.replace("\n", "")

    s = re.sub(r"([0-9\.]+\s)", add_comma, s)
    
    return eval(s)

df.residual = df.residual.apply(comma)

In [2]:
include = df

bids = list(include.residual)
covariates = [[cov] for cov in list(include.ispolice)]
incremented = list(include.increment_residual)

In [3]:
median_upper, median_lower = estimate_median(bids, covariates, incremented, (-4,6))

calculating values for covariate: [0.0] (1/2)
total time elapsed: 0.00016829899999981635s
calculating values for covariate: [1.0] (2/2)
total time elapsed: 54.299329552s


In [4]:
def loss_function(c):
    a, b = c
    cef = lambda cov: a+b*cov[0]
    return get_loss_function(covariates, median_upper, median_lower, cef)

point, interval = get_estimates(loss_function, 2)

print(point)
print(interval)

[-0.37628797  0.25954741]
[-0.4529125125423831, -0.28024178433269115, 0.029576626375221435, 0.5757825108346618]


In [5]:
cef = lambda c, cov: c[0]+c[1]*cov[0]
n_cov = 2

results = bootstrap(include, bids, covariates, incremented, [n_cov], [cef], 500)

calculating values for covariate: [0.0] (1/2)
total time elapsed: 0.0001224840000020322s
calculating values for covariate: [1.0] (2/2)
total time elapsed: 56.500091733999994s
calculating values for covariate: [0.0] (1/2)
total time elapsed: 0.000101491000009446s
calculating values for covariate: [1.0] (2/2)
total time elapsed: 52.517564608999976s
calculating values for covariate: [0.0] (1/2)
total time elapsed: 0.0005110169999511527s
calculating values for covariate: [1.0] (2/2)
total time elapsed: 52.488668866999944s
calculating values for covariate: [0.0] (1/2)
total time elapsed: 0.00011949199995342497s
calculating values for covariate: [1.0] (2/2)
total time elapsed: 50.98456061099995s
calculating values for covariate: [0.0] (1/2)
total time elapsed: 0.00010377099999914208s
calculating values for covariate: [1.0] (2/2)
total time elapsed: 50.72286538499998s
calculating values for covariate: [0.0] (1/2)
total time elapsed: 9.901500004616537e-05s
calculating values for covariate: [1.

KeyboardInterrupt: 

In [ ]:
a_lower, a_upper, b_lower, b_upper = [[a[0][i] for a in results] for i in range(4)]
print(f"a cf: [{np.percentile(a_lower, 5)}, {np.percentile(a_upper, 95)}]")
print(f"b cf: [{np.percentile(b_lower, 5)}, {np.percentile(b_upper, 95)}]")